In [2]:
import numpy as np
import matplotlib.pyplot as plt
import gym
from gym.envs.registration import register
import time
import os

In [3]:
EPOCHS = 20000
ALPHA = 0.8
GAMMA = 0.9

epsilon = 1.0
min_epsilon = 0.01
max_epsilon = 1.0
decay_val = 0.0045

In [4]:
try:
    register(
        id='FrozenLakeNotSlippery-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name':'4x4', 'is_slippery': False},
        max_episode_steps=100,
        reward_threshold=0.78, # optimum = .8196
    )
except:
    print('Already Registered')

In [5]:
env = gym.make('FrozenLakeNotSlippery-v0')

In [6]:
qTable = np.zeros([env.observation_space.n, env.action_space.n])

In [7]:
def egas(epsilon, qTable, discrete_state):
    rn = np.random.random()
    # exploitation
    if rn > epsilon:
        # print(discrete_state)
        # print('Exploit')
        state_row = qTable[discrete_state, :]
        action = np.argmax(state_row)
        # print(qTable)
    else:
        # print('random')
        action = env.action_space.sample()

    return action

def computeQValue(currentQvalue, reward, nextOptimalQValue):
    # if reward>0:
    #
    #     print(f"computeQValue : {currentQvalue + ALPHA*(reward+GAMMA*nextOptimalQValue-currentQvalue)}")

    return currentQvalue + ALPHA*(reward+GAMMA*nextOptimalQValue-currentQvalue)

def decay(epsilon, EPOCH):
    return min_epsilon+(max_epsilon-min_epsilon)*np.exp(-decay_val*EPOCH)

In [8]:
rewards = []
interval = 1000

In [10]:
for episode in range(EPOCHS):
    # print(episode)
    state = env.reset()
    # print(f"first state {state}")
    terminated = False
    totalRewards = 0

    while not terminated:
        #ACTION
        if type(state) is not int:
            state = state[0]
        action = egas(epsilon, qTable, state)

        #Get values such as state, reward, done, info
        new_state, reward, terminated, truncated, info = env.step(action)
        # if reward>0:
        #     print(f"new_state: {new_state}, reward: {reward}, terminated: {terminated}, truncated: {truncated}, info: {info}")

        #Get Current Q Value

        currentQValue = qTable[state, action]
        # print(currentQValue)

        #Get Optimal q value
        nextOptimalQValue = np.max(qTable[new_state, :])
        # if nextOptimalQValue > 0:
        #     print(f"nextOptimalQValue : {nextOptimalQValue}")

        #Compute next Q Value
        nextQValue = computeQValue(currentQValue, reward, nextOptimalQValue)
        # if reward>0:
        #     print(reward)
        # if nextQValue > 0:
        #
        #     print(nextQValue)

        #Update the table
        qTable[state, action] = nextQValue
        # if nextQValue > 0:
        #     print(qTable)

        #Track Rewards
        totalRewards = totalRewards+reward
        # print(totalRewards)

        #Update State

        state = new_state
        # print(state)
        # print("-----------------------------------------------------------")
    # print(f"this obs {state}")
    episode+=1
    epsilon = decay(epsilon, episode)
    # print(epsilon)
    rewards.append(totalRewards)
    if episode % interval == 0:
        print(np.sum(rewards))
        # pass
    # print(f"reward {np.sum(rewards)}")

env.close()

19302.0
20291.0
21276.0
22267.0
23250.0
24241.0
25231.0
26218.0
27207.0
28199.0
29195.0
30186.0
31174.0
32163.0
33150.0
34137.0
35123.0
36111.0
37100.0
38085.0


In [11]:
qTable

array([[0.531441  , 0.59049   , 0.4782969 , 0.531441  ],
       [0.531441  , 0.        , 0.43046721, 0.4782969 ],
       [0.4782969 , 0.46861286, 0.37791386, 0.43030037],
       [0.42966822, 0.        , 0.        , 0.        ],
       [0.59049   , 0.6561    , 0.        , 0.531441  ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.7985952 , 0.        , 0.34437377],
       [0.        , 0.        , 0.        , 0.        ],
       [0.6561    , 0.        , 0.729     , 0.59049   ],
       [0.6561    , 0.81      , 0.81      , 0.        ],
       [0.729     , 0.9       , 0.        , 0.63284198],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.81      , 0.9       , 0.729     ],
       [0.81      , 0.9       , 1.        , 0.81      ],
       [0.        , 0.        , 0.        , 0.        ]])

In [13]:
env = gym.make('FrozenLakeNotSlippery-v0', render_mode="human")

In [1]:
state = env.reset()

for steps in range(100):
    env.render()
    if type(state) is not int:
            state = state[0]
    action = np.argmax(qTable[state, :])

    state, reward, terminated, truncated, info = env.step(action)
    time.sleep(0.5)
    if terminated:
        env.close()

NameError: name 'env' is not defined